In [1]:
pip install transformers scikit-learn pandas numpy datasets torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm



/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pip install ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 879.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
#  from .autonotebook import tqdm as notebook_tqdm

ImportError: attempted relative import with no known parent package

In [6]:


# Load the "md_gender_bias" dataset from Hugging Face
from datasets import load_dataset

dataset = load_dataset("md_gender_bias")



In [8]:


# Extract data and labels
data = dataset["train"]["text"]
labels = dataset["train"]["labels"]



Current columns in the dataset: ['text', 'original', 'labels', 'class_type', 'turker_gender', 'episode_done', 'confidence']

In [9]:


# Data preprocessing and feature extraction for SVM and Random Forest
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_data = vectorizer.fit_transform(data)
tfidf_data = tfidf_data.toarray()




In [10]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, labels, test_size=0.2, random_state=42)



In [11]:


# Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)



/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:


# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)



/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [13]:


# Accuracy for SVM and Random Forest
svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

# Print SVM and Random Forest accuracy
print(f"SVM Accuracy: {svm_accuracy}")
print(f"Random Forest Accuracy: {rf_accuracy}")



SVM Accuracy: 0.3837953091684435
Random Forest Accuracy: 0.36673773987206826


In [14]:


# ANN and RNN models require tokenization using Transformers library
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the data
tokenized_data = tokenizer(data, padding=True, truncation=True, return_tensors="pt")

# Label encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data for ANN and RNN
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(tokenized_data.input_ids,
                                                                    encoded_labels,
                                                                    test_size=0.2,
                                                                    random_state=42)

# Artificial Neural Network (ANN) model
class ANNModel(nn.Module):
    def _init_(self):
        super(ANNModel, self)._init_()
        self.embedding = nn.Embedding(30522, 768)
        self.fc1 = nn.Linear(768, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)  # Average pooling
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

ann_model = ANNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ann_model.parameters(), lr=0.001)

# Training loop for ANN
for epoch in range(5):
    ann_model.train()
    optimizer.zero_grad()
    outputs = ann_model(X_train_ann)
    loss = criterion(outputs, y_train_ann)
    loss.backward()
    optimizer.step()

# RNN model
class RNNModel(nn.Module):
    def _init_(self):
        super(RNNModel, self)._init_()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 2)

    def forward(self, x):
        outputs = self.bert(x)
        x = outputs.last_hidden_state[:, 0, :]
        x = self.fc(x)
        return x

rnn_model = RNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)

# Training loop for RNN
for epoch in range(5):
    rnn_model.train()
    optimizer.zero_grad()
    outputs = rnn_model(input_ids=X_train_ann)
    loss = criterion(outputs, y_train_ann)
    loss.backward()
    optimizer.step()

# ANN and RNN evaluation
ann_model.eval()
ann_outputs = ann_model(X_test_ann)
ann_predictions = torch.argmax(ann_outputs, dim=1)

rnn_model.eval()
rnn_outputs = rnn_model(input_ids=X_test_ann)
rnn_predictions = torch.argmax(rnn_outputs, dim=1)

# Convert predictions to numpy arrays
ann_predictions = ann_predictions.cpu().numpy()
rnn_predictions = rnn_predictions.cpu().numpy()

# Decode labels
decoded_ann_predictions = label_encoder.inverse_transform(ann_predictions)
decoded_rnn_predictions = label_encoder.inverse_transform(rnn_predictions)

# Calculate accuracy for ANN and RNN
ann_accuracy = accuracy_score(y_test_ann, ann_predictions)
rnn_accuracy = accuracy_score(y_test_ann, rnn_predictions)

# Print ANN and RNN accuracy
print(f"ANN Accuracy: {ann_accuracy}")
print(f"RNN Accuracy: {rnn_accuracy}")

/home/codespace/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: optimizer got an empty parameter list